# BGE-M3 Korean Embedding Fine-tuning (Azure ML Studio)

이 노트북은 합성 데이터 생성 → 데이터 자산 등록 → 실험/실행 제출 → 모델 등록 → 엔드포인트 배포를 단계적으로 수행합니다.

필수 환경 변수:
- AZURE_SUBSCRIPTION_ID
- AZURE_RESOURCE_GROUP
- AZUREML_WORKSPACE_NAME
- AZUREML_COMPUTE_NAME (예: cpu-cluster)

In [3]:
import sys
from pathlib import Path

# Add parent directory to path to import src module
notebook_dir = Path().resolve()
project_root = notebook_dir.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import os

from azure.ai.ml import MLClient, command, Input
from azure.ai.ml.entities import Data, AmlCompute, ManagedOnlineEndpoint, ManagedOnlineDeployment, Model
from azure.identity import DefaultAzureCredential

from src.synth_data import read_terms, synthesize_examples, save_jsonl

from dotenv import load_dotenv

load_dotenv()

subscription_id = os.environ.get('AZURE_SUBSCRIPTION_ID')
resource_group = os.environ.get('AZURE_RESOURCE_GROUP')
workspace_name = os.environ.get('AZUREML_WORKSPACE_NAME')
compute_name = os.environ.get('AZUREML_COMPUTE_NAME', 'cpu-cluster')

assert subscription_id and resource_group and workspace_name, '환경 변수를 확인하세요.'


## 1) 합성 데이터 생성

In [2]:
terms = read_terms(Path('../data/terms_electronics_ko.txt'))
records = synthesize_examples(terms, n_pairs_per_term=6, seed=42)
output_path = Path('../data/synthetic_electronics.jsonl')
save_jsonl(records, output_path)
len(records)

966

## 2) Azure ML Client 생성

In [4]:
# Azure CLI 경로를 명시적으로 설정하여 인증 문제 해결
import os
import subprocess
import shutil

# Azure CLI 경로가 PATH에 있는지 확인하고 추가
az_cli_path = '/opt/homebrew/bin'
if az_cli_path not in os.environ.get('PATH', ''):
    os.environ['PATH'] = f"{az_cli_path}:{os.environ.get('PATH', '')}"
    print(f"Added {az_cli_path} to PATH")

# Azure CLI 접근 가능 여부 확인
az_location = shutil.which('az')
print(f"Azure CLI found at: {az_location}")

if az_location:
    # Azure CLI를 통해 인증 상태 확인
    result = subprocess.run(['az', 'account', 'show'], capture_output=True, text=True)
    if result.returncode == 0:
        print("✓ Azure CLI authentication is active")
    else:
        print("✗ Azure CLI not authenticated. Run 'az login' in terminal")

# 이제 AzureCliCredential 임포트
from azure.identity import AzureCliCredential

credential = AzureCliCredential()

ml_client = MLClient(
    credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name,
)
print(f"✓ ML Client created for workspace: {workspace_name}")

Azure CLI found at: /opt/homebrew/bin/az
✓ Azure CLI authentication is active


Class DeploymentTemplateOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


✓ ML Client created for workspace: andyaml


## 3) 데이터 자산 등록

In [5]:
data_asset = Data(
    name='bge-m3-kr-synth-train',
    version='5',
    type='uri_file',
    path=str(output_path),
)
data_asset = ml_client.data.create_or_update(data_asset)
data_asset

Data({'path': 'azureml://subscriptions/e0493f49-bc5c-4207-a643-9b5f6503a36d/resourcegroups/aistudio/workspaces/andyaml/datastores/workspaceblobstore/paths/LocalUpload/938ff3b466df609131a62c73e91ee2981ecb00d27a43a3d5156093cdc8a824a8/synthetic_electronics.jsonl', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'bge-m3-kr-synth-train', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/e0493f49-bc5c-4207-a643-9b5f6503a36d/resourceGroups/aistudio/providers/Microsoft.MachineLearningServices/workspaces/andyaml/data/bge-m3-kr-synth-train/versions/5', 'Resource__source_path': '', 'base_path': '/Users/andy/works/ai/finetuning-embedding/notebooks', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x119f31350>, 'serialize': <msrest.serialization.Serializer object at 0x119f38850>, 've

## 4) 컴퓨트 클러스터 준비

In [6]:
try:
    ml_client.compute.get(compute_name)
    print('Compute exists:', compute_name)
except Exception:
    compute = AmlCompute(name=compute_name, size='Standard_DS3_v2', min_instances=0, max_instances=2)
    ml_client.compute.begin_create_or_update(compute).result()
    print('Compute created:', compute_name)

Compute exists: clustercpu


## 5) 실험(Experiment) 및 실행(Run) 제출

In [ ]:
from azure.ai.ml.entities import Environment, UserIdentityConfiguration

custom_env = Environment(
    name="bge-m3-finetune-env",
    description="Custom environment for BGE-M3 Korean embedding fine-tuning",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    conda_file={
        "name": "bge-m3-finetune",
        "channels": ["pytorch", "conda-forge", "defaults"],
        "dependencies": [
            "python=3.10",
            "pip",
            {"pip": [
                "sentence-transformers>=2.5.0",
                "torch>=2.1.0",
                "datasets>=2.18.0",
                "accelerate>=1.1.0",
                "mlflow>=2.10.0",
                "numpy>=1.26.0",
                "scikit-learn>=1.3.0",
                "tqdm>=4.66.0",
            ]}
        ]
    }
)

# Register environment
registered_env = ml_client.environments.create_or_update(custom_env)
print(f"✓ Environment registered: {registered_env.name}:{registered_env.version}")
print(f"  Environment ID: {registered_env.id}")

job = command(
    experiment_name='bge-m3-kr-finetune',
    code='../.',
    command='python -m src.train --train_data ${{inputs.train_data}} --output_dir ${{outputs.model_output}} --epochs 1 --batch_size 16 --lr 2e-5 --max_seq_length 256',
    inputs={
        'train_data': Input(type='uri_file', path=data_asset.id),
    },
    outputs={
        'model_output': {'type': 'uri_folder'},
    },
    environment=registered_env.id,
    compute=compute_name,
    identity=UserIdentityConfiguration(),  # 사용자 인증 정보를 compute cluster에 전달
)
print("Job environment:", job.environment)
print("Job identity:", job.identity)
created_job = ml_client.jobs.create_or_update(job)
created_job

In [26]:
ml_client.jobs.stream(created_job.name)

RunId: silly_wheel_bbd0pbps14
Web View: https://ml.azure.com/runs/silly_wheel_bbd0pbps14?wsid=/subscriptions/e0493f49-bc5c-4207-a643-9b5f6503a36d/resourcegroups/aistudio/workspaces/andyaml

Execution Summary
RunId: silly_wheel_bbd0pbps14
Web View: https://ml.azure.com/runs/silly_wheel_bbd0pbps14?wsid=/subscriptions/e0493f49-bc5c-4207-a643-9b5f6503a36d/resourcegroups/aistudio/workspaces/andyaml



## 6) 모델 등록

In [28]:
model = Model(
    name='bge-m3-kr-embedding-model',
    path=f"azureml://jobs/{created_job.name}/outputs/model_output",
    type='custom_model',
)
registered_model = ml_client.models.create_or_update(model)
registered_model

Model({'job_name': 'silly_wheel_bbd0pbps14', 'intellectual_property': None, 'system_metadata': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'bge-m3-kr-embedding-model', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/e0493f49-bc5c-4207-a643-9b5f6503a36d/resourceGroups/aistudio/providers/Microsoft.MachineLearningServices/workspaces/andyaml/models/bge-m3-kr-embedding-model/versions/1', 'Resource__source_path': '', 'base_path': '/Users/andy/works/ai/finetuning-embedding/notebooks', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x11cf6ab10>, 'serialize': <msrest.serialization.Serializer object at 0x11cd52290>, 'version': '1', 'latest_version': None, 'path': 'azureml://subscriptions/e0493f49-bc5c-4207-a643-9b5f6503a36d/resourceGroups/aistudio/workspaces/andyaml/datastores/workspaceblobstore/paths/azureml/silly_wheel_bbd0pbps14/model_output/', 'datastore': None

## 7) 엔드포인트 및 배포

In [33]:
# 기존 배포가 실패했다면 삭제 후 재배포
try:
    ml_client.online_deployments.begin_delete(
        name='blue',
        endpoint_name='bge-m3-kr-embeddings'
    ).result()
    print("기존 배포 삭제 완료")
except Exception as e:
    print(f"배포 삭제 중 오류 (무시 가능): {e}")

기존 배포 삭제 완료


### 로컬 배포 테스트 (선택사항)

Azure에 배포하기 전에 로컬에서 먼저 테스트하면 디버깅이 빠릅니다.

In [ ]:
# 스코어링 스크립트를 로컬에서 간단히 테스트
# (Azure 배포 전 빠른 검증용)

# 로컬 모델 경로 설정
import os
os.environ['AZUREML_MODEL_DIR'] = str(Path('../outputs/bge-m3-kr').resolve())

# score.py 임포트 및 테스트
import sys
import importlib
score_module = importlib.import_module('src.score')

# init 함수 테스트
try:
    score_module.init()
    print("✓ init() 성공 - 모델 로드 완료")
    
    # run 함수 테스트
    import json
    test_input = json.dumps({
        "inputs": {
            "data": ["테스트 문장입니다", "또 다른 테스트"]
        }
    })
    result = score_module.run(test_input)
    result_obj = json.loads(result)
    print(f"✓ run() 성공 - {len(result_obj['embeddings'])}개 임베딩 생성")
    print(f"  임베딩 차원: {len(result_obj['embeddings'][0])}")
    print("\n로컬 테스트 성공! Azure 배포 진행 가능")
except Exception as e:
    print(f"✗ 로컬 테스트 실패: {e}")
    import traceback
    traceback.print_exc()

INFO:src.score:AZUREML_MODEL_DIR: /Users/andy/works/ai/finetuning-embedding/outputs/bge-m3-kr
INFO:src.score:Contents of AZUREML_MODEL_DIR: ['model.safetensors', '2_Normalize', '1_Pooling', 'tokenizer_config.json', 'config.json', 'config_sentence_transformers.json', 'tokenizer.json', 'README.md', 'sentence_bert_config.json', 'modules.json']
INFO:src.score:Found subdirectory: 2_Normalize
INFO:src.score:Contents: []
INFO:src.score:Loading model from: /Users/andy/works/ai/finetuning-embedding/outputs/bge-m3-kr
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: /Users/andy/works/ai/finetuning-embedding/outputs/bge-m3-kr


Loading weights:   0%|          | 0/391 [00:00<?, ?it/s]

INFO:src.score:Model loaded successfully
INFO:src.score:Received data: {'inputs': {'data': ['테스트 문장입니다', '또 다른 테스트']}}
INFO:src.score:Generating embeddings for 2 texts


✓ init() 성공 - 모델 로드 완료


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✓ run() 성공 - 2개 임베딩 생성
  임베딩 차원: 1024

로컬 테스트 성공! Azure 배포 진행 가능


INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://westus2.livediagnostics.monitor.azure.com/QuickPulseService.svc/ping?api-version=REDACTED&ikey=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '309'
    'x-ms-qps-transmission-time': 'REDACTED'
    'x-ms-qps-machine-name': 'REDACTED'
    'x-ms-qps-instance-name': 'REDACTED'
    'x-ms-qps-stream-id': 'REDACTED'
    'x-ms-qps-role-name': 'REDACTED'
    'x-ms-qps-invariant-version': 'REDACTED'
    'x-ms-qps-configuration-etag': 'REDACTED'
    'Accept': 'application/json'
A body is sent with the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Server': 'Microsoft-IIS/10.0'
    'x-ms-qps-subscribed': 'REDACTED'
    'Request-Context': 'REDACTED'
    'Access-Control-Expose-Headers': 'REDACTED'
    'X-Powered-By': 'REDACTED'
    'Strict-Transport-Security': 'REDACTED'
    'X-Content-Type-Options': 'RE

In [ ]:
from azure.ai.ml.entities import CodeConfiguration

# 추론(inference)용 환경 생성 - azureml-inference-server-http 필수
inference_env = Environment(
    name="bge-m3-inference-env",
    description="Inference environment for BGE-M3 Korean embedding model",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    conda_file={
        "name": "bge-m3-inference",
        "channels": ["pytorch", "conda-forge", "defaults"],
        "dependencies": [
            "python=3.10",
            "pip",
            {"pip": [
                "azureml-inference-server-http",
                "sentence-transformers>=2.5.0",
                "torch>=2.1.0",
                "numpy>=1.26.0",
            ]}
        ]
    }
)
registered_inference_env = ml_client.environments.create_or_update(inference_env)
print(f"✓ Inference env registered: {registered_inference_env.name}:{registered_inference_env.version}")

endpoint = ManagedOnlineEndpoint(
    name='bge-m3-kr-embeddings',
    auth_mode='key',
)
ml_client.begin_create_or_update(endpoint).result()

deployment = ManagedOnlineDeployment(
    name='blue',
    endpoint_name=endpoint.name,
    model=registered_model.id,
    environment=registered_inference_env.id,
    instance_type='Standard_DS3_v2',
    instance_count=1,
    code_configuration=CodeConfiguration(
        code='../.',
        scoring_script='src/score.py',
    ),
)
ml_client.begin_create_or_update(deployment).result()

ml_client.online_endpoints.begin_traffic_update(
    name=endpoint.name, traffic={'blue': 100}
).result()
print("✓ 배포 완료!")

INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://management.azure.com/subscriptions/e0493f49-bc5c-4207-a643-9b5f6503a36d/resourceGroups/aistudio/providers/Microsoft.MachineLearningServices/workspaces/andyaml/environments/bge-m3-inference-env?api-version=REDACTED'
Request method: 'GET'
Request headers:
    'Accept': 'application/json'
    'x-ms-client-request-id': '9a469e52-06fd-11f1-a458-136bcd49c228'
    'User-Agent': 'azure-ai-ml/1.31.0 azsdk-python-mgmt-machinelearningservices/0.1.0 Python/3.11.8 (macOS-26.2-arm64-arm-64bit)'
    'Authorization': 'REDACTED'
    'traceparent': '00-38e80474d6678221fd9c93747a1728d5-a59e2ed9cef34b8d-01'
No body was attached to the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Cache-Control': 'no-cache'
    'Pragma': 'no-cache'
    'Content-Length': '752'
    'Content-Type': 'application/json; charset=utf-8'
    'Expires': '-1'
    'Vary': 'REDACTED'
    'x-ms-ratelimi

✓ Inference env registered: bge-m3-inference-env:1


INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://westus2-0.in.applicationinsights.azure.com//v2.1/track'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '7907'
    'Accept': 'application/json'
    'x-ms-client-request-id': '9d22e36a-06fd-11f1-a458-136bcd49c228'
    'User-Agent': 'azsdk-python-azuremonitorclient/unknown Python/3.11.8 (macOS-26.2-arm64-arm-64bit)'
A body is sent with the request
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://westus2-0.in.applicationinsights.azure.com//v2.1/track'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '8190'
    'Accept': 'application/json'
    'x-ms-client-request-id': '9d3afb8a-06fd-11f1-a458-136bcd49c228'
    'User-Agent': 'azsdk-python-azuremonitorclient/unknown Python/3.11.8 (macOS-26.2-arm64-arm-64bit)'
A body is sent with the request
INFO:azure.core.pipeline.policies.http_logging

KeyboardInterrupt: 

INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://westus2-0.in.applicationinsights.azure.com//v2.1/track'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '4768'
    'Accept': 'application/json'
    'x-ms-client-request-id': 'a7d437a0-06fd-11f1-a458-136bcd49c228'
    'User-Agent': 'azsdk-python-azuremonitorclient/unknown Python/3.11.8 (macOS-26.2-arm64-arm-64bit)'
A body is sent with the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json; charset=utf-8'
    'Server': 'Microsoft-HTTPAPI/2.0'
    'Strict-Transport-Security': 'REDACTED'
    'X-Content-Type-Options': 'REDACTED'
    'Date': 'Wed, 11 Feb 2026 03:56:33 GMT'
INFO:azure.monitor.opentelemetry.exporter.export._base:Transmission succeeded: Item received: 7. Items accepted: 7
INFO:azure.core.pipeline.policies.http_logging_policy:Reque

In [ ]:
# 배포 로그 확인 (실패 원인 파악용)
try:
    logs = ml_client.online_deployments.get_logs(
        name='blue',
        endpoint_name='bge-m3-kr-embeddings',
        lines=100
    )
    print("=== Deployment Logs ===")
    print(logs)
except Exception as e:
    print(f"로그 조회 실패: {e}")

INFO:azure.identity._internal.decorators:AzureCliCredential.get_token_info succeeded
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://management.azure.com/subscriptions/e0493f49-bc5c-4207-a643-9b5f6503a36d/resourceGroups/aistudio/providers/Microsoft.MachineLearningServices/workspaces/andyaml/onlineEndpoints/bge-m3-kr-embeddings/deployments/blue/getLogs?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '13'
    'Accept': 'application/json'
    'x-ms-client-request-id': '556a094a-06fd-11f1-a458-136bcd49c228'
    'User-Agent': 'azure-ai-ml/1.31.0 azsdk-python-mgmt-machinelearningservices/0.1.0 Python/3.11.8 (macOS-26.2-arm64-arm-64bit)'
    'Authorization': 'REDACTED'
    'traceparent': '00-790bc9bdc823abb7179761f0ae9c8da0-2548407191ffe3a0-01'
A body is sent with the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Cache-Control': 'no

=== Deployment Logs ===
Instance status:
SystemSetup: Succeeded
UserContainerImagePull: Succeeded
ModelDownload: Succeeded
UserContainerStart: Succeeded

Container events:
Kind: Pod, Name: Downloading, Type: Normal, Time: 2026-02-11T02:45:16.176558Z, Message: Start downloading models
Kind: Pod, Name: Pulling, Type: Normal, Time: 2026-02-11T02:45:16.186298Z, Message: Start pulling container image
Kind: Pod, Name: Pulled, Type: Normal, Time: 2026-02-11T02:48:41.549954Z, Message: Container image is pulled successfully
Kind: Pod, Name: Downloaded, Type: Normal, Time: 2026-02-11T02:48:41.549954Z, Message: Models are downloaded successfully
Kind: Pod, Name: Created, Type: Normal, Time: 2026-02-11T02:48:41.939361Z, Message: Created container inference-server
Kind: Pod, Name: Started, Type: Normal, Time: 2026-02-11T02:48:42.233114Z, Message: Started container inference-server
Kind: Pod, Name: ContainerReady, Type: Normal, Time: 2026-02-11T02:49:02.494416838Z, Message: Container is ready

Conta

INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://westus2-0.in.applicationinsights.azure.com//v2.1/track'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '4410'
    'Accept': 'application/json'
    'x-ms-client-request-id': '5821e2a2-06fd-11f1-a458-136bcd49c228'
    'User-Agent': 'azsdk-python-azuremonitorclient/unknown Python/3.11.8 (macOS-26.2-arm64-arm-64bit)'
A body is sent with the request
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://westus2-0.in.applicationinsights.azure.com//v2.1/track'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '2762'
    'Accept': 'application/json'
    'x-ms-client-request-id': '582eee5c-06fd-11f1-a458-136bcd49c228'
    'User-Agent': 'azsdk-python-azuremonitorclient/unknown Python/3.11.8 (macOS-26.2-arm64-arm-64bit)'
A body is sent with the request
INFO:azure.core.pipeline.policies.http_logging

## 8) 엔드포인트 테스트

배포된 엔드포인트에 샘플 요청을 보내 정상 작동을 확인합니다.

In [ ]:
import json

# 배포 디렉토리 생성
deploy_dir = Path('deploy')
deploy_dir.mkdir(exist_ok=True)

# 샘플 요청 데이터 생성 (임베딩 모델용)
sample_request = {
    "inputs": {
        "data": [
            "전자제품 추천 부탁드립니다",
            "노트북 구매 고려중입니다",
            "스마트폰 배터리 수명이 궁금합니다"
        ]
    }
}

# JSON 파일로 저장
sample_request_path = deploy_dir / 'sample-request.json'
with open(sample_request_path, 'w', encoding='utf-8') as f:
    json.dump(sample_request, f, ensure_ascii=False, indent=2)

print(f"샘플 요청 파일 생성: {sample_request_path}")
print(json.dumps(sample_request, ensure_ascii=False, indent=2))

샘플 요청 파일 생성: deploy/sample-request.json
{
  "inputs": {
    "data": [
      "전자제품 추천 부탁드립니다",
      "노트북 구매 고려중입니다",
      "스마트폰 배터리 수명이 궁금합니다"
    ]
  }
}


In [ ]:
# 엔드포인트 호출 테스트
response = ml_client.online_endpoints.invoke(
    endpoint_name=endpoint.name,
    request_file=str(sample_request_path),
    deployment_name='blue'
)

print("응답 결과:")
print(response)

INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://management.azure.com/subscriptions/e0493f49-bc5c-4207-a643-9b5f6503a36d/resourceGroups/aistudio/providers/Microsoft.MachineLearningServices/workspaces/andyaml/onlineEndpoints/bge-m3-kr-embeddings/deployments?api-version=REDACTED'
Request method: 'GET'
Request headers:
    'Accept': 'application/json'
    'x-ms-client-request-id': 'aa2bcb94-06fd-11f1-a458-136bcd49c228'
    'User-Agent': 'azure-ai-ml/1.31.0 azsdk-python-mgmt-machinelearningservices/0.1.0 Python/3.11.8 (macOS-26.2-arm64-arm-64bit)'
    'Authorization': 'REDACTED'
    'traceparent': '00-ddaa3c1ff6379ae9cb8f0ae2a31256d9-4830bcaae4441efb-01'
No body was attached to the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Cache-Control': 'no-cache'
    'Pragma': 'no-cache'
    'Content-Length': '2357'
    'Content-Type': 'application/json; charset=utf-8'
    'Expires': '-1'
    'Vary': 'REDACTED'
  

응답 결과:
"{\"embeddings\": [[-0.030284546315670013, 0.03242814540863037, -0.05122189223766327, -0.007127018645405769, 0.008139829151332378, -0.0024081759620457888, 0.010629973374307156, -0.0077257235534489155, 0.004988341126590967, -0.02222408354282379, 0.07664605975151062, 0.04373449832201004, -0.030558958649635315, -0.02916059084236622, 0.008973916992545128, -0.01672874204814434, -0.014758817851543427, -0.02286103181540966, 0.016925232484936714, -0.048317890614271164, 0.009380176663398743, 0.039821039885282516, -0.015478244051337242, 0.034162282943725586, 0.011927800253033638, -0.0047517637722194195, -0.03580474108457565, -0.03495727851986885, 0.06955274194478989, -0.011221639811992645, -0.023940619081258774, -0.05325997993350029, -0.012763358652591705, -0.05084911361336708, -0.03565908223390579, -0.006706611718982458, -0.006046571768820286, -0.06568180024623871, -0.07486022263765335, 0.025724228471517563, -0.006193569861352444, 0.01914861984550953, 0.021467939019203186, -0.02628556266

INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://westus2-0.in.applicationinsights.azure.com//v2.1/track'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '1427'
    'Accept': 'application/json'
    'x-ms-client-request-id': 'af8b1ab8-06fd-11f1-a458-136bcd49c228'
    'User-Agent': 'azsdk-python-azuremonitorclient/unknown Python/3.11.8 (macOS-26.2-arm64-arm-64bit)'
A body is sent with the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json; charset=utf-8'
    'Server': 'Microsoft-HTTPAPI/2.0'
    'Strict-Transport-Security': 'REDACTED'
    'X-Content-Type-Options': 'REDACTED'
    'Date': 'Wed, 11 Feb 2026 03:56:46 GMT'
INFO:azure.monitor.opentelemetry.exporter.export._base:Transmission succeeded: Item received: 1. Items accepted: 1
INFO:azure.core.pipeline.policies.http_logging_policy:Reque

## 9) 리소스 정리

비용 절감을 위해 사용하지 않는 리소스를 정리합니다.

In [ ]:
# 엔드포인트 삭제 (필요시 주석 해제)
# ml_client.online_endpoints.begin_delete(name=endpoint.name).result()
# print(f"엔드포인트 '{endpoint.name}' 삭제 완료")

# 컴퓨팅 클러스터는 자동으로 스케일 다운되므로 별도 삭제 불필요
# 하지만 완전히 삭제하려면:
# ml_client.compute.begin_delete(compute_name).result()
# print(f"컴퓨팅 클러스터 '{compute_name}' 삭제 완료")